# Upload a table to the sandbox, match format to existing table

In [11]:
import sys
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
sys.path.append("C:/Users/ehbaker/Documents/Python/Repos/ice2O") #Path to where DBImport.py is saved
import DbImport #This is a module that I have written, stored one directory up (cd ..)
import numpy as np
import settings

In [12]:
#User-supplied criteria:
db_table='draw_wire' #name of table in the database which you want to copy
pth=(r"Q:\Project Data\GlacierData\WOLVERINE\Draw_Wire\data\processed\draw_wire_database.csv") #path to csv for upload
df=pd.read_csv(pth) #read to dataframe

In [13]:
#Connect to the database
cs=settings.import_cs() #user-defined module to store connection info
engine = create_engine('postgresql://' + cs['user'] + ':' + str(cs['password']) + '@' + cs['host'] + ':' + cs['port'] + '/' + cs['dbname'])

In [14]:
#Extract format from existing table
types=DbImport.define_db_table_format(df, db_table, engine)

In [15]:
types

,attname,type
0,station_name,text
1,extension,double precision
2,datetime,timestamp without time zone
3,gid,integer


In [16]:
#Check to see if the columns in the new data frame match the ones in the database (order not important)
columns_match=set(list(types['attname'])) ==set(list(df))
columns_match

True

In [17]:
#Connect to sandbox
engine_sand = create_engine('postgresql://' + cs['user'] + ':' + str(cs['password']) + '@' + cs['host'] + ':' + cs['port'] + '/' + 'sandbox')

In [9]:
#Add subset of new table to sandbox
df=df.sample(30) #overwrite table with a subset, as a test
sandbox_tab_name=db_table +'_temp' #name for new table in sandbox portion of DB
if columns_match==True:
    df.to_sql(name=sandbox_tab_name, con=engine_sand, index = False, if_exists='replace')
else: print("ERROR: columns in uploaded table do not match those in DB")

In [18]:
#Extract format from existing table in database
types=DbImport.define_db_table_format(df, db_table, engine)
types

,attname,type
0,station_name,text
1,extension,double precision
2,datetime,timestamp without time zone
3,gid,integer


In [20]:
#Reformat columns in sandbox to match format of those in existing table in spatial_database
for xx in range(0,types.shape[0]):
    col_name=types.attname[xx]
    col_type=types.type[xx]
    query=(r"""ALTER TABLE %s
     ALTER COLUMN %s TYPE %s
     USING %s::%s""")%(sandbox_tab_name, col_name, col_type, col_name, col_type)
    engine_sand.execute(query)
    print("done with " +col_name)
print("DONE with" + sandbox_tab_name)

done with station_name
done with extension
done with datetime
done with gid


### Done! You have uploaded a table to the sandbox which matches structure of the original table (columns and formats)